In [ ]:
import json, pycm, itertools, pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from scipy.stats import spearmanr

In [ ]:
experiments = json.load(open('ablation_study_model-agnostic_2.json', 'r'))

In [ ]:
ablation_metrics = {}
for experiment in experiments:
    df1 = pd.DataFrame.from_records(experiment["data"])
    df1a = df1[["label", "p(Hallucination)"]].rename(columns={'label': 'h_label', 'p(Hallucination)': 'h_p'})
    df2 = pd.DataFrame.from_records(experiment["predictions"])
    df2a = df2[["label", "p(Hallucination)"]].rename(columns={'label': 's_label', 'p(Hallucination)': 's_p'})
    df3 = pd.concat([df1a, df2a], axis=1)
    cm = pycm.ConfusionMatrix(df3["h_label"].tolist(), df3["s_label"].tolist(), digit=2, classes=[ 'Hallucination', 'Not Hallucination' ])
    rho, _ = spearmanr(df3['h_p'], df3['s_p'])
    examples, task_defined, role_defined, hallucination_defined = (experiment["config"]["examples"], experiment["config"]["task_defined"], experiment["config"]["role_defined"], experiment["config"]["hallucination_defined"])
    if examples and task_defined and role_defined and hallucination_defined:
        ablation = "none"
    elif not examples and task_defined and role_defined and hallucination_defined:
        ablation = "w/o\nexamples"
    elif not examples and not task_defined and role_defined and hallucination_defined:
        ablation = "and w/o\ntask"
    elif not examples and not task_defined and not role_defined and hallucination_defined:
        ablation = "and w/o\nrole"
    elif not examples and not task_defined and not role_defined and not hallucination_defined:
        ablation = "and w/o\nconcept"
    if not ablation in ablation_metrics:
        ablation_metrics[ablation] = { "accuracy": [], "rho": [], "kappa": [] }
    ablation_metrics[ablation]["accuracy"].append(cm.ACC['Hallucination'])
    ablation_metrics[ablation]["rho"].append(rho)
    ablation_metrics[ablation]["kappa"].append(cm.Kappa)

In [ ]:
flattened_metrics = []

for ablation, metrics in ablation_metrics.items():
    for metric_name, values in metrics.items():
        for value in values:
            flattened_metrics.append({"Ablation": ablation, "Metric": metric_name, "Value": value})

df_long = pd.DataFrame.from_records(flattened_metrics)
df_long2 = df_long[df_long["Metric"] == "accuracy"]
df_long3 = df_long[df_long["Metric"] == "rho"]
df_long4 = df_long[df_long["Metric"] == "kappa"]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 3.5))

sns.set_theme(style="whitegrid")
sns.boxplot(x='Ablation', y='Value', data=df_long2, palette="Set2", ax=axes[0])
axes[0].set_xlabel('\nablation')
axes[0].set_ylabel('accuracy')
axes[0].set_ylim(0.72, 0.79)

sns.set_theme(style="whitegrid")
sns.boxplot(x='Ablation', y='Value', data=df_long3, palette="Set2", ax=axes[1])
axes[1].set_xlabel('\nablation')
axes[1].set_ylabel('rho')
axes[1].set_ylim(0.57, 0.65)

plt.tight_layout()
plt.show()